In [1]:
import pickle 
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

from scipy.optimize import minimize
from scipy.spatial import ConvexHull, convex_hull_plot_2d

from pymatgen.ext.matproj import MPRester
from pymatgen.core import Element, Composition
from pymatgen.analysis.phase_diagram import GrandPotentialPhaseDiagram, PhaseDiagram, PDPlotter, PDEntry

# Initialize the MP Rester
mpr = MPRester('pn8XdbGhMrv90STu')
ssf = SiteStatsFingerprint(
    CrystalNNFingerprint.from_preset('ops', distance_cutoffs=None, x_diff_weight=0),
    stats=('mean', 'std_dev', 'minimum', 'maximum'))



NameError: name 'SiteStatsFingerprint' is not defined

In [ ]:
unary_data = pickle.load(open("data_gather/unary_oxide_data.p", "rb"))
binary_oxide_data = pickle.load(open("data_gather/binary_oxide_data_form_ene.p", "rb"))
ele2gs = pickle.load(open("data_gather/ele2gs.p", "rb"))

u_correction = {
    "V": -1.700,
    "Cr": -1.999,
    "Mn": -1.668,
    "Fe": -2.256,
    "Co": -1.638,
    "Ni": -2.541,
    "Mo": -3.202,
    "W": -4.438,
}

# define your reference materials project ids you want to compare, for now taking rutile if exists, or lowest
# octahedral MO2.
ele2mp = {
    'Mo': 'mp-510536',
    'W': 'mp-19372',
    'Nb': 'mp-2533',
    'Ni': 'mp-35925', # layered Ni
    'V': 'mp-19094',
    'Ir': 'mp-2723',
    'Ru': 'mp-825',
    'Cr': 'mp-19177',
    'Fe': 'mp-850222',
    'Mn': 'mp-510408',
    'Co': 'mp-1272749', # layered Co 
    'Pb': 'mp-20725',
    'Sb': 'mvc-9819', # mvc-9819 chosen since it is 0.17 eV/atom more unstable than the most stable MP SbO2 (this is chosen because OQMD has this difference for its rutile SbO2)
    'Sn': 'mp-856',
    'Bi': 'mp-557993',
    'Ti': 'mp-2657',
    'Ta': 'mp-20994',
    'In': 'mp-1181008',
    'Pt': 'mp-1077716',
    'Pd': 'mp-1018886',
    'Re': 'mp-12875',
    'Os': 'mp-996',
    'Ge': 'mp-2633',# not rutile
    'Tc': 'mp-1205302',
    'Rh': 'mp-725',
}


d2metals = {
    '3': ['Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni'],
    '4': ['Nb', 'Mo', 'Tc', 'Ru', 'Pd', ],
    '5': ['Ta', 'W', 'Re', 'Os', 'Ir', 'Pt']
}

ele2col = {
    "Ti": "#808080",
    "V": "#ffcccb",
    "Cr": "k",
    "Mn": "y",
    "Fe": "r",
    "Co": "#00008B",
    "Cu": "#FFFF00",
    "Nb": "cyan",
    "Mo": "#FFA500",
    "Sn": "k",
    "W": "fuchsia",
    "Ge": "g",
    "Rh": "#FFA500",
    "Sb": "#66ff00",
    "Te": "y",
    "Ir": "b",
    "Ni": "#66FF00",
    "Bi": "b",
    'Sr': '#A020F0',
    'Ru': 'y',
    'Ta': '#8b0000',
    'In': '#a9a9a9',
    'Pt': 'gray',
    'Pd': 'orange',
    'Re': 'purple',
    'Os': 'brown',
    'Tc': 'indigo',
    'Pb': '#4169e1',
    'Rh': 'cyan'
} 


oqmd_quadratic_equations_ox = pickle.load(open('make_quadratics/oqmd_quadratic_equations_ox.p', 'rb'))
oqdm_quadratic_equations_red = pickle.load(open('make_quadratics/oqmd_quadratic_equations_red.p', 'rb'))

mp_quadratic_equations_ox = pickle.load(open('make_quadratics/mp_oqmd_quadratic_equations_ox.p', 'rb'))
mp_quadratic_equations_red = pickle.load(open('make_quadratics/mp_oqmd_quadratic_equations_red.p', 'rb'))


potential_hosts = ['Ag', 'Ge', 'Bi', 'Co', 'Ir', 'Nb', 'Pb', 'Pd', 'Pt', 'Rh',
                  'Ru', 'Sb', 'Si', 'Sn', 'Ta', 'Te', 'Ti', 'Tl', 'W']
potential_hosts = [ 'Co', 'Ir', 'Nb','Sb', 'W']

abundant_hosts = ['Bi', 'Co', 'Nb', 'Pb', 'Sb', 'Si', 'Sn', 'Ti', 'Tl', 'W']
easy2deposit = ['Bi', 'Pb', 'Sn', 'W']

templates = ['Pb', 'Sn', 'Sb', 'Bi', 'Ta', 'Ti', 'W', 'Nb', 'Mo', 'Ir', 'Ru']
cr_mixers = ['Nb', 'V', 'Mo', 'Ir', 'Mn', 'W']
#templates = [ 'Ta']
mn_mixers = ['Ti', 'Cr', 'V', 'Fe', 'Sn']
fe_mixers = ['Mn', 'W']

In [ ]:
def get_ref_data(element, reference_oxide_id):
    for idx, id_ in enumerate(unary_data[element]['mp_ids']):
        if id_==reference_oxide_id:
            ref_fingerprint = unary_data[element]['fingerprints'][idx]
            ref_struct = unary_data[element]['structures'][idx]
            ref_ene = unary_data[element]['energies'][idx]
            ref_form_ene = unary_data[element]['formation_energies'][idx]
            #print("Reference oxide formation energy: {:.3f} eV".format(ref_form_ene))
            return ref_form_ene, ref_fingerprint, ref_struct


def uncorr_ene_from_formation_energy(formation_energy_correction, ele1, ele2, ele1_conc, o_conc):
    """
    Return the synthetically calculated uncorrected DFT energy given a formation energy correction with
    respect to the reference oxides.
    """
    o_energy = -4.948
    o_correction = -0.687
    ele1_ene, ele1_fingerprint, ele1_struct = get_ref_data(ele1, ele2mp[ele1])
    ele2_ene, ele2_fingerprint, ele2_struct = get_ref_data(ele2, ele2mp[ele2])
    
    total_form_ene = ele1_ene*ele1_conc+ele2_ene*(1-ele1_conc)+formation_energy_correction
    end_member_ene = (1-o_conc)*(ele1_conc*ele2gs[ele1]+(1-ele1_conc)*ele2gs[ele2])+(o_conc)*(o_energy)
    corrections = o_conc*(o_correction)
    # now get the corrections that would be applied per atom
    if ele1 in u_correction.keys():
        corrections += (1-o_conc)*(ele1_conc*u_correction[ele1])
    if ele2 in u_correction.keys():
        corrections += (1-o_conc)*((1-ele1_conc)*u_correction[ele2])
    # by definition formation energy = (mixed system energy) - (end member energy), here we just manipulate
    # that equation and remove the MP corrections.
    uncorr_energy = total_form_ene+end_member_ene
    return uncorr_energy 


def make_synthetic_mp(energy_correction, ele1, ele2, ele1_conc, o_conc):
    name = "{}{}{}{}O{}".format(ele1, (1-o_conc)*(ele1_conc), ele2, (1-o_conc)*(1-ele1_conc), o_conc)
    composition = Composition(name)
    label = "{}{:.3f}{}{:.3f}O{:.3f}".format(ele1, (1-o_conc)*(ele1_conc), ele2, (1-o_conc)*(1-ele1_conc), o_conc)
    energy = uncorr_ene_from_formation_energy(energy_correction, ele1, ele2, ele1_conc, o_conc)
    return PDEntry(composition, energy, name=name), label


def get_hull_energy(min_mu_o, max_mu_o, entries):
    list_mu_Os = np.linspace(min_mu_o, max_mu_o, 10)
    data = []
    hull = []
    for mu_Os in list_mu_Os:
        open_elements_specific = {Element("O"):mu_Os}
        gcpd = GrandPotentialPhaseDiagram(entries, open_elements_specific)
        l = [gcpd.get_form_energy_per_atom(e) for e in gcpd.all_entries]
        h = [gcpd.get_e_above_hull(e) for e in gcpd.all_entries]
        data.append(l)
        hull.append(h)
    hull = np.array(hull)
    data = np.array(data)
    return hull, data, gcpd, list_mu_Os


def load_data(ele1, ele2):
    entries = []
    labels = []
    entries_1 = mpr.get_entries('{}-{}-O'.format(ele1, ele2))+mpr.get_entries('{}-O'.format(ele1))+mpr.get_entries('{}-O'.format(ele2))
    for entry in entries_1:
        if 'H' in entry.name:
            continue
        entries.append(entry)
        labels.append(entry.name)
    return entries, labels


def plot_coords(reference_element_):
    struct_differences = []
    mp_ids = []
    ox2lowest = {}
    ox2data = defaultdict(list)

    for idx, struct in enumerate(unary_data[reference_element_]['structures']):
        m_coordinations, ox_state, o_o_coords, o_m_coords, o_pc = structure_inspection(struct)
        mpid = unary_data[reference_element_]['mp_ids'][idx]
        mp_ids.append(mpid)
        query = mpr.get_entry_by_material_id(mpid, property_data=['formation_energy_per_atom',])
        formation_energy = query.data['formation_energy_per_atom']
        if set(list(m_coordinations))=={6}:
            plt.plot(ox_state, formation_energy, 'bo', alpha=0.1)
        elif set(list(m_coordinations))=={4}:
            plt.plot(ox_state, formation_energy, 'ro', alpha=0.9)
            #print(mpid)
        elif set(sorted(list(m_coordinations)))=={5}:
            plt.plot(ox_state, formation_energy, 'go', alpha=0.9)
        #else:
        #    plt.plot(ox_state, formation_energy, 'yo', alpha=0.7)

    plt.show()

def get_conc_in_binary(structure, element):
    element_count = 0
    other_element_count = 0
    for site in structure.sites:
        if str(site.specie)==element:
            element_count+=1
        elif str(site.specie)=='O':
            continue
        else:
            other_element_count+=1
    return element_count/(element_count+other_element_count)


def get_binary_mp_data(element1, element2):
    pairings = []
    found_key = False
    for key in binary_oxide_data.keys():
        if element1 in key and element2 in key:
            # the order of the elements is arbitrary, depends how the data was gathered
            needed_key = key
            found_key = True
    if not found_key:
        print("No data for combination: {}, {}".format(element1, element2))
        return 
    
    conc2lowest = defaultdict()
    lowest_ene = 0 
    for idx, struct in enumerate(binary_oxide_data[needed_key]['structures']):
        m_coords, ox_state, o_o_coords, o_m_coords, o_pc = structure_inspection(struct)
        if ox_state==4 and list(set(m_coords))==[6]:
            # then we care about this structure to make comparisons
            dope_conc = get_conc_in_binary(struct, element2)
            #print(dope_conc)
            #print(binary_oxide_data['{}_Sb'.format(dope_element)]['names'][idx])
            mixed_energy = binary_oxide_data[needed_key]['formation_energies'][idx]
            dope_o2_ene, dope_o2_fingerprint, dope_o2_struct = get_ref_data(element2, ele2mp[element2])
            sbo2_ene, sbo2_fingerprint, sbo2_struct = get_ref_data(element1, ele2mp[element1])
            ##if element2=='Ni':
             #   dope_o2_ene+=0.05
            mixing_stabilisation = mixed_energy - (dope_o2_ene*dope_conc+(sbo2_ene)*(1-dope_conc))
            pairings.append((dope_conc, mixing_stabilisation, element2))
            gs_key = element2+"_{:.2f}".format(dope_conc)
            if gs_key not in conc2lowest:
                conc2lowest[element2+"_{:.2f}".format(dope_conc)] = (dope_conc, mixing_stabilisation)
            elif conc2lowest[gs_key][1]>mixing_stabilisation:
                conc2lowest[element2+"_{:.2f}".format(dope_conc)] = (dope_conc, mixing_stabilisation)
    return pairings, conc2lowest


def formation_energy_mixing(ox_quadratic_1, red_quadratic_1, ox_quadratic_2, red_quadratic_2, concentrations):
    pairings = []
    ground_states =  []

    #concs = np.arange(0.0000001, 0.9999999, conc_interval)
    predicted_forms_1 = []
    predicted_forms_2 = []
    predicted_ox_change_1 = []
    predicted_ox_change_2 = []
    concs =[]
    synthetic_entries = []
    for conc in concentrations:
        o_1a, o_2a, o_3a = ox_quadratic_1
        r_1a, r_2a, r_3a = red_quadratic_1

        o_1b, o_2b, o_3b = ox_quadratic_2
        r_1b, r_2b, r_3b = red_quadratic_2

        result_1 = minimize(oxidation_reduction_equation, 1, args=(conc, o_1a, o_2a, o_3a, r_1a, r_2a, r_3a),  bounds=[(0, 3)])
        # Note, the definition of concentration is different  between 1/2, handle this when plotting 
        # by flipping the x-axis horizontally
        result_2 = minimize(oxidation_reduction_equation, 1, args=(conc, o_1b, o_2b, o_3b, r_1b, r_2b, r_3b),  bounds=[(0, 3)])

        predicted_ox_change_1.append(result_1.x[0])
        predicted_forms_1.append(result_1.fun[0])

        predicted_ox_change_2.append(result_2.x[0])
        predicted_forms_2.append(result_2.fun[0])
        concs.append(conc)
        
    xs_1 = []
    xs_2 = []
    for conc in concs:
        # need to shift the energies so that either end is set to 0
        # so shift every point proportionally
        xs_1.append(predicted_forms_1[0]*(1-conc)+predicted_forms_1[-1]*conc)
        xs_2.append(predicted_forms_2[0]*(1-conc)+predicted_forms_2[-1]*conc)
    
    return np.array(predicted_forms_1)-xs_1, np.array(predicted_forms_2)-xs_2, predicted_ox_change_1, predicted_ox_change_2

        

def oxidation_reduction_equation(x, ox_conc, o1, o2, o3, r1, r2, r3):
    """
    Defines the equation to optimize oxidation state when mixed
    Args:
    x: The amount of oxidation state change
    ox_conc: The concentration of the oxidising element between 0 and 1
    o1, o2, o3: Define the quadratic equation for the oxidising element
    r1, r2, r3: Define the quadratic equation for the reducing element
    """
    return (o1*x**2+o2*x+o3)+(r1*(ox_conc*x/(1-ox_conc))**2+r2*(ox_conc*x/(1-ox_conc))+r3)



In [ ]:
for housing_element in ['Sb']:
    if housing_element not in mp_quadratic_equations_ox.keys() or housing_element not in mp_quadratic_equations_red.keys():
        print("Missing eq for {}".format(housing_element))
        continue
    pairings = []
    ground_states =  []
    plt.figure(figsize=(12, 9))
    concs = np.arange(0.0000001, 0.9999999, 0.001)

    #housing_concs = np.arange(0.0000001, 0.9999999, .001)
    for ele in ['Mn']:#d2metals['3']:#+d2metals['4']+d2metals['5']:
        if ele==housing_element:
            continue
        if ele=='Ni' or ele=='Co':
            pred_1, pred_2, ox_1, ox_2 = formation_energy_mixing(oqmd_quadratic_equations_ox[ele], mp_quadratic_equations_red[housing_element],
                                                                 mp_quadratic_equations_ox[housing_element], oqmd_quadratic_equations_red[ele], concs)
        elif housing_element=='Sb':
            pred_1, pred_2, ox_1, ox_2 = formation_energy_mixing(mp_quadratic_equations_ox[ele], oqmd_quadratic_equations_red[housing_element],
                                                                 mp_oqmd_quadratic_equations_ox[housing_element], mp_quadratic_equations_red[ele], concs)
        else:
            pred_1, pred_2, ox_1, ox_2 = formation_energy_mixing(mp_quadratic_equations_ox[ele], mp_quadratic_equations_red[housing_element],
                                                                 mp_quadratic_equations_ox[housing_element], mp_quadratic_equations_red[ele], concs)


        can_reduce = True
        can_oxidise = True
        # check if it found any solutions 
        if sum(ox_1)==0:
            #print("Cant reduce {}, and oxidise {}".format(housing_element, ele))
            can_reduce = False
        if sum(ox_2)==0:
            #print("Cant oxidise {}, and reduce {}".format(housing_element, ele))
            can_oxidise = False 
            
        if can_reduce:
            #axes[int(count/3)][count%3].plot(concs, np.array(predicted_forms_1)-xs_1, label="B={}".format(ele), color=ele2col[ele])#, 'r-')
            #axes[int(count/3)][count%3].plot(concs, np.array(predicted_forms_1)-xs_1, color=ele2col[ele])#, 'r-')
            #axes[int(count/3)][count%3].plot([concs[0], concs[-1]], [0, 0], 'k--')
            #pass
            plt.plot(concs, pred_1, color=ele2col[ele])#, 'r-')
            plt.plot([concs[0], concs[-1]], [0, 0], 'k--')
        if can_oxidise:
            #axes[int(count/3)][count%3].plot(-np.array(concs)+1, np.array(predicted_forms_2)-xs_2, label="B={}".format(ele), ls='-', color=ele2col[ele])#, 'r-')
            #axes[int(count/3)][count%3].plot([concs[0], concs[-1]], [0, 0], 'k--')
            plt.plot(-np.array(concs)+1, pred_2, label="B={}".format(ele), ls='-', color=ele2col[ele])#, 'r-')
            plt.plot([concs[0], concs[-1]], [0, 0], 'k--')
            
        plt.show()
        indices = np.round(np.linspace(0, len(concs) - 1, 10)).astype(int)
        concs2check = concs[indices]
        #mp_entries, mp_labels = load_data(ele, housing_element)
        #synth_entries = []
#             if conc in concs2check:
#                 #print(min(pred_1[idx], pred_2[idx]))
#                 corr = min(pred_1[idx], pred_2[idx])
#                 print(corr)
#                 entry_, label_ = make_synthetic_mp(corr, ele, housing_element, conc, 2./3)
#                 synth_entries.append(entry_)
        
#         hull_, data_, gpcd_, mus_ = get_hull_energy(-7, -4, mp_entries+synth_entries)
#         hull = np.array(hull_)
#         data = np.array(data_)
        ##################################


        #E above hull for each material figure as a function of mu
#         plt.figure(figsize=(9, 9))
#         for i, name in enumerate(gpcd_.all_entries): #order of all_entries is not the same as the order of entries, annoying for labels...
#             if '.' in name.name or ('Mn(SbO3' in name.name):
#                 plt.plot(mus_,hull[:,i], label=name.name)
#                 print(hull[:,i])
#                 print(name)
        #verticle line at mu_O (e.g., -4.66)

#         plt.xlabel(r'$\mu_O$(eV)')
#         plt.ylabel('E above Hull (eV/MetalAtom)')
#         plt.xlim(mus_[0],mus_[-1])
#         plt.ylim(-0.05,1)
#         plt.legend(loc='center left')#, bbox_to_anchor=(1, 0.5))
#         plt.tight_layout()
#         plt.show()
                #print(conc)    print(housing_element)
    #plt.ylim(-0.54, 0.54)
    #print(housing_element)
    #plt.show()
    #plt.savefig('mnsb_hull.pdf') 
        
        